In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [4]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,2):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] 
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"GeschäftsNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [5]:
df_a=pd.DataFrame(alle_seiten)

In [6]:
df_a["GeschäftsNR"]= df_a["GeschäftsNR"].astype(int) #die Geschäftsnummer (VorlagenNR) zu Zahl machen

In [7]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df_a["url"]:
    url_liste_ganz.append(url_anfang_original+element)
    

In [8]:
#Nun seite für Seite abrufen und in einem neuen Minidict unterbringen
eingang_list=[]

for geschaeft in url_liste_ganz:
    rg=requests.get(geschaeft)
    soup_g=BeautifulSoup(rg.text, "html.parser")
    tr_g_list=soup_g.find_all("tr")
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text   
    ul_list = tr_g_list[2].find_all("ul")
    li_list = ul_list[0].find_all("li")
    
    for url in li_list:
        urlna = url.find("a")["href"]
        name = url.find("a").text
    
    
    
        minidict_g={"GeschäftsNR":gesch_nr, "Einreichedatum":einger_am, "Namens-url": urlna, "Name eingereicht":name}
        for key, value in minidict_g.items():
            if value == '':
                minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
            else:
                minidict_g[key] = value
            
        eingang_list.append(minidict_g)

                #### Wieso??? Wieso habe ich danach alle namen in vierfacher Ausgabe?

In [9]:
df_g=pd.DataFrame(eingang_list)
pd.set_option("display.max_rows", 3000)

In [10]:
df_g

,GeschäftsNR,Einreichedatum,Namens-url,Name eingereicht
0,3010,10.09.2019,https://zg-comp.clex.ch/frontend/people/145,Egger Ivo
1,3010,10.09.2019,https://zg-comp.clex.ch/frontend/people/164,Vuichard Stéphanie
2,3010,10.09.2019,https://zg-comp.clex.ch/frontend/people/64,Schriber-Neiger Hanni
3,3010,10.09.2019,https://zg-comp.clex.ch/frontend/people/58,Lustenberger Andreas
4,3009,10.09.2019,https://zg-comp.clex.ch/frontend/people/164,Vuichard Stéphanie
5,3009,10.09.2019,https://zg-comp.clex.ch/frontend/people/116,Hess Mariann
6,3009,10.09.2019,https://zg-comp.clex.ch/frontend/people/98,Hofer Rita
7,3008,10.09.2019,https://zg-comp.clex.ch/frontend/people/151,Leemann Rainer
8,3008,10.09.2019,https://zg-comp.clex.ch/frontend/people/143,Arnold Michael
9,3007,08.09.2019,https://zg-comp.clex.ch/frontend/fractions/4,SP


In [19]:
#nun versuche ich auf den Namens-Seiten die Partei sowie den Namen herauszuziehen.
liste_parteien=[]
for url_p in df_g["Namens-url"]:
    rp=requests.get(url_p)
    soup_p=BeautifulSoup(rp.text, "html.parser")
    h1_p_list=soup_p.find_all("h1")
    div_p_list=soup_p.find_all("div")
    partei_p = div_p_list[1].find_all("a")[51].text
    name_p=h1_p_list[0].text
    
    minidict_p={"Name eingereicht":name_p, "Partei":partei_p}
    liste_parteien.append(minidict_p)

In [20]:
df_p=pd.DataFrame(liste_parteien)

In [21]:
df_p #### Bullshit - die Namen und vorallem die Parteien sind da nicht immer am selben Ort:-( zumindest nicht von hinten.
#müssen wir anders schauen. Vielleicht anhand des Untertitels "Parteien" (h2) navigieren? ist das möglich?

,Name eingereicht,Partei
0,Egger Ivo,ALG
1,Egger Ivo,ALG
2,Egger Ivo,ALG
3,Egger Ivo,ALG
4,Vuichard Stéphanie,ALG
5,Vuichard Stéphanie,ALG
6,Vuichard Stéphanie,ALG
7,Vuichard Stéphanie,ALG
8,Schriber-Neiger Hanni,18.12.2014 - 14.12.2018 (2015-2018)
9,Schriber-Neiger Hanni,18.12.2014 - 14.12.2018 (2015-2018)


In [ ]:
df_g["GeschäftsNR"]= df_g["GeschäftsNR"].astype(int) #verwandle string in Zahl
df_g["Einreichedatum"] =  pd.to_datetime(df_g["Einreichedatum"], format='%d.%m.%Y') #verwandle string in datum

In [ ]:
df_mit_dat=pd.merge(df_a, df_g, how="left", on="GeschäftsNR") #ich verbinde die beiden DataFrames

In [ ]:
df_mit_dat

In [10]:
#zuerst mit einer einzigen Namens-url arbeiten
rp=requests.get("https://zg-comp.clex.ch/frontend/people/164")
soup_p=BeautifulSoup(rp.text, "html.parser")
h1_p_list=soup_p.find_all("h1")
name_p=h1_p_list[0].text
div_p_list=soup_p.find_all("div")
partei_p = div_p_list[1].find_all("a")[-8].text

In [17]:
a_list_p=div_p_list[1].find_all("a")[51]

In [18]:
a_list_p

<a href="https://zg-comp.clex.ch/frontend/parties/5">ALG</a>

In [ ]:
#hier der Ort der Partei:
partei_p=li_p_list[6].text
#und hier nochmals ihr Name um die Sach zu verknüpfen.
name_p=li_p_list[-9].text